# Lab 8: Greedy Algorithms

Lab associated with Module 8: Greedy Algorithms

***

In [1]:
# The following lines are used to increase the width of cells to utilize more space on the screen 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\benny\AppData\Local\Temp\ipykernel_35256\1945220536.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


***

### Section 0: Imports

In [2]:
import numpy as np

In [3]:
import math

In [4]:
from IPython.display import Image
from graphviz import Digraph

***

### Node and Graph Classes

In [5]:
import numpy as np

class Node:
    
    def __init__(self, v):

        self.value = v
        self.inNeighbors = []
        self.outNeighbors = []
        
        self.status = "unvisited"
        self.estD = np.inf

        self.parent = None
        
    def hasOutNeighbor(self, v):
        
        if v in self.outNeighbors:
            return True
        
        return False
        
    def hasInNeighbor(self, v):
        
        if v in self.inNeighbors:
            return True
        
        return False
    
    def hasNeighbor(self, v):
        
        if v in self.inNeighbors or v in self.outNeighbors:
            return True
        
        return False
    
    def getOutNeighbors(self):
        
        return self.outNeighbors
    
    def getInNeighbors(self):
        
        return self.inNeighbors
    
    def getOutNeighborsWithWeights(self):
        
        return self.outNeighbors
    
    def getInNeighborsWithWeights(self):
        
        return self.inNeighbors
    
    # ------------------------------------------------
    # Let us modify following two functions to incorporate weights
    # ------------------------------------------------
    
    def addOutNeighbor(self,v,wt):
        
        self.outNeighbors.append((v,wt))
    
    def addInNeighbor(self,v,wt):
        
        self.inNeighbors.append((v,wt))
        
        
    def __str__(self):
        
        return str(self.value) 


class Graph:
    
    def __init__(self):
        
        self.vertices = []

    def addVertex(self,n):
        
        self.vertices.append(n)
        
    # ------------------------------------------------
    # Let us modify following two functions to incorporate weights
    # ------------------------------------------------
        
    def addDiEdge(self, u, v, wt = 1):
        
        u.addOutNeighbor(v, wt = wt)
        v.addInNeighbor(u, wt = wt)
        
    # add edges in both directions between u and v
    def addBiEdge(self, u, v, wt = 1):
        
        self.addDiEdge(u, v, wt = wt)
        self.addDiEdge(v, u, wt = wt)
        
            
    # get a list of all the directed edges
    # directed edges are a list of two vertices
    def getDirEdges(self):
        
        ret = []
        for v in self.vertices:
            ret += [ [v, u] for u in v.outNeighbors ]
        return ret
    
    # reverse the edge between u and v.  Multiple edges are not supported.
    def reverseEdge(self,u,v):
        
        if u.hasOutNeighbor(v) and v.hasInNeighbor(u):
            
            if v.hasOutNeighbor(u) and u.hasInNeighbor(v): 
                return
        
            self.addDiEdge(v, u)
            u.outNeighbors.remove(v)
            v.inNeighbors.remove(u)        
                
    def __str__(self):
        
        ret = "Graph with:\n"
        ret += "\t Vertices:\n\t"
        for v in self.vertices:
            ret += str(v) + ","
        ret += "\n"
        ret += "\t Edges:\n\t"
        for a,b in self.getDirEdges():
            ret += "(" + str(a) + "," + str(b) + ") "
        ret += "\n"
        return ret

### <font color='red'> Activity 1a: Write a greedy algorithm for Activity Selection problem in this week's lab. </font>

![Union is appending](Union-is-appending.png)

### Realised that Union is Appending to the list, not adding to the set
### Making sure the list is sorted before returning it changed the result

In [6]:
# activity selection algorithm
# activites a1, a2 ... an
# start time s1, s2 ... sn
# finish time f1, f2 ... fn
# we have to do n activities and each activity has to be started at a specified time and finished at specified time
# if the activities overlap, can we devise an algorithm to come-up with an order on the activities to do as many activities as possible
# The question is which activities should we choose and which should we skip
# sorted finish time is O(n) otherwise O(nlogn)
# each step is locally optimal

def greedyActivitySelection(activites):
    if len(activites) == 0:
        return []
    sortedActivities = sorted(activites, key=lambda x: x[1])
    selectedActivities = [sortedActivities[0]]
    for i in range(1, len(sortedActivities)):
        if sortedActivities[i][0] >= selectedActivities[-1][1]:
            selectedActivities.append(sortedActivities[i])
    return selectedActivities


In [7]:
activities = [  [1,4],[2,5],[3,6],[5,7],[3,8],[6,9],[8,10],[9,11],[5,12],[6,13],[8,14],[13,15] ]

activityList = greedyActivitySelection(activities)

print("Solution:")
for activity in activityList:
    print(activity)

Solution:
[1, 4]
[5, 7]
[8, 10]
[13, 15]


In [8]:
import unittest

class TestGreedyActivitySelection(unittest.TestCase):
    
    def test_empty_activities(self):
        activities = []
        expected_output = []
        self.assertEqual(greedyActivitySelection(activities), expected_output)
        
    def test_single_activity(self):
        activities = [(1, 2)]
        expected_output = [(1, 2)]
        self.assertEqual(greedyActivitySelection(activities), expected_output)
        
    def test_disjoint_activities(self):
        activities = [(1, 2), (3, 4), (5, 6)]
        expected_output = [(1, 2), (3, 4), (5, 6)]
        self.assertEqual(greedyActivitySelection(activities), expected_output)
        
    def test_overlapping_activities(self):
        activities = [(1, 3), (2, 4), (3, 5), (4, 6)]
        expected_output_1 = [(1, 3), (3, 5)]
        expected_output_2 = [(2, 4), (4, 6)]
        result = greedyActivitySelection(activities)
        self.assertTrue(result in [expected_output_1, expected_output_2])

# Create a test suite
suite = unittest.TestLoader().loadTestsFromTestCase(TestGreedyActivitySelection)

# Run the test suite and print the results
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_disjoint_activities (__main__.TestGreedyActivitySelection.test_disjoint_activities) ... ok
test_empty_activities (__main__.TestGreedyActivitySelection.test_empty_activities) ... ok
test_overlapping_activities (__main__.TestGreedyActivitySelection.test_overlapping_activities) ... ok
test_single_activity (__main__.TestGreedyActivitySelection.test_single_activity) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

### <font color='red'> Activity 1b: Devise a dynamic programming solution to Activity Selection problem, and see if you get the same results as that of greedy strategy. </font>

In [21]:
# a dynamic programming solution to the activity selection problem
def dynamicProgrammingActivitySelection(activities):
    #test for empty list
    if len(activities) == 0:
        return []
    sorted_activities = sorted(activities, key=lambda x: x[1])
    # initialize the DP table
    DP = [0] * len(sorted_activities)
    DP[0] = 1
    for i in range(1, len(sorted_activities)):
        # find the last compatible activity
        j = i - 1
        while j >= 0 and sorted_activities[j][1] > sorted_activities[i][0]:
            j -= 1
        # update the DP table
        DP[i] = max(DP[i - 1], DP[j] + 1)
    # construct a solution
    solution = []
    i = len(sorted_activities) - 1
    while i >= 0:
        if i > 0 and DP[i] == DP[i - 1]:
            i -= 1
        else:
            solution.append(sorted_activities[i])
            i = j

    return solution[::-1]
        







Let us test above function as:

In [22]:
activities = [  [1, 2], [3, 4], [5, 6] ]

solution = dynamicProgrammingActivitySelection(activities)

print(f"Solution has {len(solution)} activities:")
for i in solution:
    print(i, end=" ")


Solution has 2 activities:
[1, 2] [5, 6] 

***

In [11]:
import unittest

class TestDynamicProgrammingActivitySelection(unittest.TestCase):

    def test_no_activities(self):
        activities = []
        solution = dynamicProgrammingActivitySelection(activities)
        self.assertEqual(solution, [], "Test Case 1 Failed")

    def test_single_activity(self):
        activities = [[1, 4]]
        solution = dynamicProgrammingActivitySelection(activities)
        self.assertEqual(solution, [[1, 4]], "Test Case 2 Failed")

    def test_multiple_non_overlapping_activities(self):
        activities = [[1, 2], [3, 4], [5, 6]]
        solution = dynamicProgrammingActivitySelection(activities)
        self.assertEqual(solution, [[1, 2], [3, 4], [5, 6]], "Test Case 3 Failed")

    def test_multiple_overlapping_activities_original_list(self):
        activities = [[1, 4], [2, 5], [3, 6], [5, 7], [3, 8], [6, 9], [8, 10], [9, 11], [5, 12], [6, 13], [8, 14], [13, 15]]
        solution = dynamicProgrammingActivitySelection(activities)
        # Expected solution depends on your implementation

    def test_multiple_overlapping_activities_different_scenario(self):
        activities = [[1, 4], [2, 6], [5, 7], [6, 8]]
        solution = dynamicProgrammingActivitySelection(activities)
        # Expected solution depends on your implementation

    def test_activities_with_same_start_time(self):
        activities = [[1, 4], [1, 3], [2, 5], [3, 6]]
        solution = dynamicProgrammingActivitySelection(activities)
        # Expected solution depends on your implementation

    def test_activities_with_same_end_time(self):
        activities = [[1, 4], [2, 4], [3, 4]]
        solution = dynamicProgrammingActivitySelection(activities)
        # Expected solution depends on your implementation

# Create a test suite
suite = unittest.TestLoader().loadTestsFromTestCase(TestDynamicProgrammingActivitySelection)

# Run the test suite and print the results
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)


test_activities_with_same_end_time (__main__.TestDynamicProgrammingActivitySelection.test_activities_with_same_end_time) ... ok
test_activities_with_same_start_time (__main__.TestDynamicProgrammingActivitySelection.test_activities_with_same_start_time) ... ok
test_multiple_non_overlapping_activities (__main__.TestDynamicProgrammingActivitySelection.test_multiple_non_overlapping_activities) ... FAIL
test_multiple_overlapping_activities_different_scenario (__main__.TestDynamicProgrammingActivitySelection.test_multiple_overlapping_activities_different_scenario) ... ok
test_multiple_overlapping_activities_original_list (__main__.TestDynamicProgrammingActivitySelection.test_multiple_overlapping_activities_original_list) ... ok
test_no_activities (__main__.TestDynamicProgrammingActivitySelection.test_no_activities) ... ok
test_single_activity (__main__.TestDynamicProgrammingActivitySelection.test_single_activity) ... ok

FAIL: test_multiple_non_overlapping_activities (__main__.TestDynamicPro

<unittest.runner.TextTestResult run=7 errors=0 failures=1>

### This image shows how to calculate the weighted sum of completion times

![Weighted Sum](weighted-sum.png)

###  Prim's Algorithm

Graph's Preliminaries

In [12]:
from graph import *

ModuleNotFoundError: No module named 'graph'

In [ ]:
G = Graph()

for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    G.addVertex( Node(i) )

In [ ]:
V = G.vertices

#0, 1, 2, 3, 4, 5, 6, 7, 8
#A, B, C, D, E, F, G, H, I

G.addBiEdge( V[0], V[1], 4)

G.addBiEdge( V[0], V[7] , 8)

G.addBiEdge( V[1], V[7], 11)

G.addBiEdge( V[1], V[2], 8)

G.addBiEdge( V[2], V[3], 7)

G.addBiEdge( V[3], V[4], 9)

G.addBiEdge( V[3], V[5], 14 )

G.addBiEdge( V[4], V[5], 10 )

G.addBiEdge( V[2], V[5], 4 )

G.addBiEdge( V[2], V[8], 2 )

G.addBiEdge( V[5], V[6], 2 )

G.addBiEdge( V[6], V[7], 1 )

G.addBiEdge( V[6], V[8], 6 )

G.addBiEdge( V[7], V[8], 7 )


In [ ]:
print(G)

This is what we had in the lectures as the slow implementation of Prim's Algorithm

In [ ]:
# G is graph
# s is the node to start

def slowPrim(G, s):
    
    # first, find the lightest edge leaving s
    bestWt = np.inf
    bestu = None
    
    for u,wt in s.getOutNeighborsWithWeights():
        
        if wt < bestWt:
            bestWt = wt
            bestu = u
    
    MST = [ (s, bestu) ]
    verticesVisited = [s,bestu]
    
    while len(verticesVisited) < len(G.vertices): # danger! this will loop forever if the graph isn't connected...
        
        # find the lightest edge (x,v) so that x has been visited and v hasn't.
        bestWt = np.inf
        bestv = None
        bestx = None
        
        for x in verticesVisited:
            for v,wt in x.getOutNeighborsWithWeights():
                
                if v in verticesVisited:
                    continue
                
                if wt < bestWt:
                    bestWt = wt
                    bestv = v
                    bestx = x
                    
        MST.append((bestx,bestv))
        verticesVisited.append(bestv)
    
    return MST

In [ ]:
T = slowPrim(G, G.vertices[0])

for x,y in T:
    print(x,y)

Okay, it seems to be working fine, but as we discussed, will be quite slow. Let us see if we can work on the faster version of the code as:

### <font color='red'> Activity 2: In lights of Prim's Algorithm above, write an efficient implementation based on our discussions in the Seminar/Lecture. </font>

In [ ]:
def prim(G,w):
    
    #### TODO ####
    ### Good Luck ###
                    
    return MST

In [ ]:
T = prim(G, G.vertices[0])

for x,y in T:
    print(x,y)

***